# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [2]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [3]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'
#url = 'https://www.zalando.fr/api/catalog/articles?campaigns=fr_ss19_eoss_wave_2_fs_summer_cgs_2019_25&categories=enfant&limit=84&offset=84&sort=sale'

#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [4]:
# headers definition
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}


In [5]:
# Your code
response = requests.get(url,headers = headers)
results = response.json()
results

{'total_count': 20269,
 'pagination': {'page_count': 242, 'current_page': 2, 'per_page': 84},
 'sort': 'sale',
 'articles': [{'sku': 'NI114F002-A13',
   'name': 'FORCE 1 18 - Chaussures premiers pas - white/indigo force/blue gaze',
   'price': {'original': '49,95\xa0€',
    'promotional': '35,00\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['19.5', '23.5', '25', '26', '27'],
   'url_key': 'nike-sportswear-force-1-18-baskets-basses-ni114f002-a13',
   'media': [{'path': 'NI/11/4F/00/2A/13/NI114F002-A13@5.jpg',
     'role': 'DEFAULT',
     'packet_shot': False}],
   'brand_name': 'Nike Sportswear',
   'is_premium': False,
   'family_articles': [{'sku': 'NI114F002-A13',
     'url_key': 'nike-sportswear-force-1-18-baskets-basses-ni114f002-a13',
     'media': [{'path': 'NI/11/4F/00/2A/13/NI114F002-A13@2.jpg',
       'role': 'FAMILY',
       'p

#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [6]:
# Get the total number of pages
page_count = results["pagination"]["page_count"]
print(page_count)
# Your code
flattened_data = json_normalize(results)
flattened_data #Mostramos el json para ver que forma tiene


242


,articles,articlesToShow,breadcrumbs,carouselTeaser,categoryTree,collection,contentPositions.entry-point-teasers,contentPositions.in-cat-carousel,contentPositions.in-cat-carousel-fullwidth,contentPositions.in-cat-carousel-mobile,...,total_article_count,total_count,upperInCatTeaser,variants.fullWidthCatalog,variants.hideCategories,variants.mobileLightFilters,variants.myBrandsFilter,variants.outwardTeaserCard,variants.premiumCatalog,wishlist
0,"[{'sku': 'NI114F002-A13', 'name': 'FORCE 1 18 ...",84,"[{'items': [{'label': 'Enfant', 'url_key': 'en...",None,"[{'label': 'Promotions', 'id': '9574', 'url_ke...",None,"[7, 14, 20, 26]",9,8,6,...,20268,20269,None,False,False,False,True,False,False,None


In [7]:
flattened_articles =  json_normalize(flattened_data.articles[0])
flattened_articles
#Como solo nos interesa el "articles", volvemos a hacer un flatten para mostrar los resultados

,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,sku,url_key
0,Nike Sportswear,"[{'sku': 'NI114F002-A13', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'NI/11/4F/00/2A/13/NI114F002-A13@5.j...,FORCE 1 18 - Chaussures premiers pas - white/i...,False,False,False,False,"49,95 €","35,00 €",shoe,"[19.5, 23.5, 25, 26, 27]",NI114F002-A13,nike-sportswear-force-1-18-baskets-basses-ni11...
1,MOSCHINO,"[{'sku': '6MO24K008-K11', 'url_key': 'moschino...","[{'key': 'discountRate', 'value': '-70%', 'tra...",True,[{'path': '6M/O2/4K/00/8K/11/6MO24K008-K11@7.j...,HOODED - Sweatshirt - baya blue maramao,False,True,True,False,"159,95 €","47,95 €",clothing,"[4a, 6a, 8a, 10a, 12a, 14a]",6MO24K008-K11,moschino-hooded-sweatshirt-baya-blue-maramao-6...
2,Nike Performance,"[{'sku': 'N1243G09X-Q11', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': 'Jusqu’à -45...",False,[{'path': 'N1/24/3G/09/XQ/11/N1243G09X-Q11@4.j...,veste en sweat zippée - black/sail/university ...,False,True,True,False,"54,95 €","29,95 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",N1243G09X-Q11,nike-performance-air-hoodie-veste-en-sweat-zip...
3,Nike Sportswear,"[{'sku': 'NI114D05L-A12', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",False,[{'path': 'NI/11/4D/05/LA/12/NI114D05L-A12@12....,COURT ROYALE (PSV) - Baskets basses - white,False,True,True,False,"39,95 €","28,00 €",shoe,"[27.5, 28.5, 29.5, 30, 31, 31.5, 32, 33, 33.5,...",NI114D05L-A12,nike-sportswear-court-royale-baskets-basses-ni...
4,Re-Gen,"[{'sku': 'RG524K00Q-K11', 'url_key': 're-gen-t...","[{'key': 'discountRate', 'value': '-70%', 'tra...",False,[{'path': 'RG/52/4K/00/QK/11/RG524K00Q-K11@5.j...,TEENS - Pullover - blue nights,False,False,False,False,"29,95 €","9,00 €",clothing,"[10a, 12a, 14a, 16a]",RG524K00Q-K11,re-gen-teens-pullover-blue-nights-rg524k00q-k11
5,Friboo,"[{'sku': 'F5724G03E-Q11', 'url_key': 'friboo-t...","[{'key': 'discountRate', 'value': '-70%', 'tra...",False,[{'path': 'F5/72/4G/03/EQ/11/F5724G03E-Q11@3.j...,T-shirt imprimé - anthracite,False,True,True,False,"12,95 €","3,90 €",clothing,"[5-6a, 7-8a, 9-10a, 11-12a, 13-14a]",F5724G03E-Q11,friboo-t-shirt-imprime-anthracite-f5724g03e-q11
6,Nike Performance,"[{'sku': 'N1243E0J0-C11', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': '-50%', 'tra...",False,[{'path': 'N1/24/3E/0J/0C/11/N1243E0J0-C11@5.j...,AIR PANT - Pantalon de survêtement - dark grey...,False,False,False,False,"44,95 €","22,48 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",N1243E0J0-C11,nike-performance-air-pant-pantalon-de-survetem...
7,Lacoste,"[{'sku': 'LA224G016-M11', 'url_key': 'lacoste-...","[{'key': 'discountRate', 'value': 'Jusqu’à -32...",True,[{'path': 'LA/22/4G/01/6M/11/LA224G016-M11@6.j...,BASIC - Polo - roquette,False,True,True,False,"44,00 €","30,00 €",clothing,"[1a, 2a, 3y, 4a, 6a, 8a, 10a, 12a, 14a, 16a]",LA224G016-M11,lacoste-polo-roquette-la224g016-m11
8,GAP,"[{'sku': 'GP024J01Q-T11', 'url_key': 'gap-todd...","[{'key': 'discountRate', 'value': '-70%', 'tra...",False,[{'path': 'GP/02/4J/01/QT/11/GP024J01Q-T11@2.j...,TODDLER BOY CRAZY CREW - Pullover - multicolor,False,False,False,False,"24,95 €","7,50 €",clothing,"[2a, 3a, 4a, 5a, 12-18m, 18-24m]",GP024J01Q-T11,gap-toddler-boy-crazy-crew-pullover-multicolor...
9,Nike Performance,"[{'sku': 'N1243E0IP-K12', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'N1/24/3E/0I/PK/12/N1243E0IP-K12@2.j...,FAVORITES AIR - Collants - blue void/white,False,False,False,False,"24,95 €","17,46 €",clothing,"[8-10a, 10-12a, 12-13a, 13-15a]",N1243E0IP-K12,nike-performance-favorites-air-collants-n1243e...


In [8]:
data_zalando = pd.DataFrame()
#Creamos un df vacio
#Creamos un for para repetir el proceso de request para introducir cada pagina en el df
for page in range (page_count):
    url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=' + str(page*84) +'&sort=sale'
    response = requests.get(url,headers = headers)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_articles =  json_normalize(flattened_data.articles[0])
    flattened_articles = flattened_articles.set_index('sku')
    data_zalando = data_zalando.append(flattened_articles)
    
    #Ahora añadimos a la df

/home/martin/.local/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
data_zalando.head()

,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
sku,,,,,,,,,,,,,,,,,,,
N1243D11U-Q11,NaN,Nike Performance,"[{'sku': 'N1243D11U-Q11', 'url_key': 'nike-per...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'N1/24/3D/11/UQ/11/N1243D11U-Q11@12....,DRY - T-shirt imprimé - black/white,False,True,True,False,"24,95 €","19,95 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=6f...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,nike-performance-cr7-dry-t-shirt-de-sport-blac...
NI114D09A-Q13,NaN,Nike Sportswear,"[{'sku': 'NI114D09A-Q13', 'url_key': 'nike-spo...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'NI/11/4D/09/AQ/13/NI114D09A-Q13@6.j...,AIR MAX 720 - Baskets basses - grey/orange,False,True,True,False,"149,95 €","90,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",https://ccp-et.metrigo.zalan.do/event/sbv?z=6f...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,nike-sportswear-air-max-720-baskets-basses-ni1...
N1243L00A-Q11,NaN,Nike Performance,"[{'sku': 'N1243L00A-Q11', 'url_key': 'nike-per...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'N1/24/3L/00/AQ/11/N1243L00A-Q11@11....,SKIRT AIR - Jupe de sport - black/white,False,False,False,False,"34,95 €","28,00 €",clothing,"[8-10a, 10-12a, 12-13a, 13-15a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=6f...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=6...,ccp,nike-performance-skirt-air-jupe-de-sport-black...
NI114F00V-C11,NaN,Nike Sportswear,"[{'sku': 'NI114F00V-C11', 'url_key': 'nike-spo...","[{'key': 'campaign', 'value': 'HOT DROP', 'tra...",False,[{'path': 'NI/11/4F/00/VC/11/NI114F00V-C11@4.j...,REACT PRESTO - Chaussures premiers pas - wolf ...,False,True,True,False,"59,95 €","24,00 €",shoe,"[21, 22, 23.5, 25, 26, 27]",NaN,NaN,NaN,nike-sportswear-nike-react-presto-baskets-bass...
CO423G015-G11,NaN,Converse,"[{'sku': 'CO423G015-G11', 'url_key': 'converse...","[{'key': 'discountRate', 'value': '-70%', 'tra...",False,[{'path': 'CO/42/3G/01/5G/11/CO423G015-G11@8.j...,STAR TRIM TEE - T-shirt imprimé - enamel red,False,False,False,False,"28,95 €","8,70 €",clothing,"[8-10a, 10-12a, 12-13a, 13-15a]",NaN,NaN,NaN,converse-star-trim-tee-t-shirt-imprime-co423g0...


In [10]:
data_zalando.shape

(20271, 19)

In [2]:
#Creo un data frame nuevo con el que trabajar:
zalando = data_zalando

NameError: name 'data_zalando' is not defined

#### Display the trending brand in DataFrame

In [12]:
# your code
trending_brands = zalando['brand_name'].value_counts()
trending_brands.head(1) #Mostramos la marca mas repetida

Friboo    857
Name: brand_name, dtype: int64

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [3]:
#Our data is still text. Convert prices into numbers.
zalando['price.original'] = (zalando[['price.original']].replace('[\€,]', '', regex=True).astype(float))/100
zalando['price.promotional'] = (zalando[['price.promotional']].replace('[\€,]', '', regex=True).astype(float))/100
zalando['discounts'] = zalando['price.original'] - zalando['price.promotional']

zalando.head()

NameError: name 'zalando' is not defined

In [14]:
discounts = zalando.groupby('brand_name',as_index=False)['discounts'].sum()
discounts.sort_values('discounts', ascending=False).head(1)

,brand_name,discounts
81,Friboo,9380.73


#### Display the brands without discount at all

In [15]:
# your code
no_discounts = zalando.groupby('brand_name',as_index=False)['discounts'].sum()
no_discounts = no_discounts[no_discounts['discounts']==0]
no_discounts.head()
#no me sale ninguna marca sin descuentos. Eso es porque, al menos un producto de cada marca tiene descuentos.

,brand_name,discounts
259,Teva,0.0


In [ ]:
#No digo nada, al siguiente dia que lo lancé si salio una marca sin nignun descuento.